In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.types
# from pyspark import SparkConf
from pyspark import pandas
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, FloatType
# from pyspark.sql.functions import pandas_udf
import re
from pyspark.sql import functions
import math
from datetime import datetime
import time

In [2]:
# _sconf = SparkConf().setAppName("practice").setMaster("locals")
_context = SparkContext("local", "practice")
spark = SparkSession(_context)

22/12/28 12:02:10 WARN Utils: Your hostname, babydata resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/12/28 12:02:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/28 12:02:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
path_data = '../data/Merging.csv'

In [4]:
_schema = StructType([\
            StructField("Flight_date", StringType(), True), \
            StructField("From", StringType(), True), \
            StructField("To", StringType(), True), \
            StructField("Duration", StringType(), True), \
            StructField("STD", StringType(), True), \
            StructField("ATD", StringType(), True), \
            StructField("STA", StringType(), True), \
            StructField("ATA", StringType(), True),\
            StructField("Signal", StringType(), True),\
            StructField("Temp", IntegerType(), True),\
            StructField("Weather", StringType(), True),\
            StructField("Wind", IntegerType(), True),\
            StructField("Direction", StringType(), True),\
            StructField("Humidity", IntegerType(), True),\
            StructField("Barometer", IntegerType(), True),\
            StructField("Visibility", StringType(), True)])

In [5]:
_data = spark.read.option("delimiter", ",").option("header", True).schema(_schema).csv(path_data)

<h1><font color="red">Tien xu ly du lieu</font></h1>

## <i>XU LY GIA TRI NULL\NONE</i>

In [6]:
_data.explain()

== Physical Plan ==
FileScan csv [Flight_date#0,From#1,To#2,Duration#3,STD#4,ATD#5,STA#6,ATA#7,Signal#8,Temp#9,Weather#10,Wind#11,Direction#12,Humidity#13,Barometer#14,Visibility#15] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/hong_hai/BigData/Code/Final_Project/data/Merging.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Flight_date:string,From:string,To:string,Duration:string,STD:string,ATD:string,STA:string,...




In [7]:
_data.printSchema()

root
 |-- Flight_date: string (nullable = true)
 |-- From: string (nullable = true)
 |-- To: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- STD: string (nullable = true)
 |-- ATD: string (nullable = true)
 |-- STA: string (nullable = true)
 |-- ATA: string (nullable = true)
 |-- Signal: string (nullable = true)
 |-- Temp: integer (nullable = true)
 |-- Weather: string (nullable = true)
 |-- Wind: integer (nullable = true)
 |-- Direction: string (nullable = true)
 |-- Humidity: integer (nullable = true)
 |-- Barometer: integer (nullable = true)
 |-- Visibility: string (nullable = true)



In [8]:
_data.select([functions.count( functions.when( functions.isnan(column) | functions.col(column).isNull(), column)).alias(column) for column in _data.columns]).show()

+-----------+----+---+--------+---+---+---+---+------+----+-------+----+---------+--------+---------+----------+
|Flight_date|From| To|Duration|STD|ATD|STA|ATA|Signal|Temp|Weather|Wind|Direction|Humidity|Barometer|Visibility|
+-----------+----+---+--------+---+---+---+---+------+----+-------+----+---------+--------+---------+----------+
|          0|   0|  0|       0|  0|  0|  0|  0|     0|8317|   8322|9226|     8317|    8326|     8378|     78156|
+-----------+----+---+--------+---+---+---+---+------+----+-------+----+---------+--------+---------+----------+



In [9]:
_listColNull = ['Temp', 'Weather', 'Wind', 'Direction', 'Humidity', 'Barometer']

In [10]:
Clear_data_1 = _data.where(_data['Temp'].isNotNull())

In [11]:
Clear_data_1.select([functions.count( functions.when( functions.isnan(column) | functions.col(column).isNull(), column)).alias(column) for column in Clear_data_1.columns]).show(truncate=False)

+-----------+----+---+--------+---+---+---+---+------+----+-------+----+---------+--------+---------+----------+
|Flight_date|From|To |Duration|STD|ATD|STA|ATA|Signal|Temp|Weather|Wind|Direction|Humidity|Barometer|Visibility|
+-----------+----+---+--------+---+---+---+---+------+----+-------+----+---------+--------+---------+----------+
|0          |0   |0  |0       |0  |0  |0  |0  |0     |0   |5      |909 |0        |9       |61       |69839     |
+-----------+----+---+--------+---+---+---+---+------+----+-------+----+---------+--------+---------+----------+



In [12]:
Clear_data_1.count()

100521

In [13]:
Clear_data_2 = Clear_data_1.drop("Visibility")

In [14]:
Clear_data_2.columns

['Flight_date',
 'From',
 'To',
 'Duration',
 'STD',
 'ATD',
 'STA',
 'ATA',
 'Signal',
 'Temp',
 'Weather',
 'Wind',
 'Direction',
 'Humidity',
 'Barometer']

In [15]:
Clear_data_2.groupBy("Wind").agg(functions.count("Wind").alias("num_row")).sort(functions.col("num_row").desc()).show(Clear_data_2.count(),truncate=False)

+----+-------+
|Wind|num_row|
+----+-------+
|7   |13816  |
|6   |13548  |
|9   |13101  |
|11  |11132  |
|4   |10833  |
|13  |9459   |
|15  |7467   |
|17  |5857   |
|2   |5297   |
|19  |3908   |
|20  |2099   |
|22  |1246   |
|24  |635    |
|26  |443    |
|28  |310    |
|30  |183    |
|32  |96     |
|33  |79     |
|35  |33     |
|41  |22     |
|39  |18     |
|37  |12     |
|43  |11     |
|46  |4      |
|5   |1      |
|21  |1      |
|50  |1      |
|null|0      |
+----+-------+



In [16]:
fill_data_1 = Clear_data_2.fillna(13816, "Wind")

In [17]:
fill_data_1.groupBy("Humidity").agg(functions.count("Humidity").alias("num_row")).sort(functions.col("num_row").desc()).show(fill_data_1.count(),truncate=False)

+--------+-------+
|Humidity|num_row|
+--------+-------+
|89      |9105   |
|94      |8861   |
|79      |8734   |
|84      |7976   |
|70      |6171   |
|66      |4752   |
|59      |4595   |
|74      |4098   |
|75      |3879   |
|49      |3514   |
|100     |3500   |
|62      |3283   |
|63      |3149   |
|55      |3124   |
|52      |2861   |
|67      |2505   |
|56      |2222   |
|46      |2007   |
|44      |1571   |
|41      |1230   |
|53      |1166   |
|47      |1068   |
|71      |1065   |
|58      |1032   |
|83      |1027   |
|43      |741    |
|36      |738    |
|51      |669    |
|38      |592    |
|39      |529    |
|34      |463    |
|78      |444    |
|65      |439    |
|50      |425    |
|48      |423    |
|69      |381    |
|32      |303    |
|61      |255    |
|54      |229    |
|40      |224    |
|30      |156    |
|45      |143    |
|73      |109    |
|37      |107    |
|42      |103    |
|88      |101    |
|28      |93     |
|57      |78     |
|33      |74     |
|35      |58

In [18]:
fill_data_2 = fill_data_1.fillna(9105, "Humidity")

In [19]:
fill_data_2.groupBy("Barometer").agg(functions.count("Barometer").alias("num_row")).sort(functions.col("num_row").desc()).show(fill_data_2.count(),truncate=False)

+---------+-------+
|Barometer|num_row|
+---------+-------+
|1008     |15193  |
|1009     |14569  |
|1010     |12998  |
|1007     |12504  |
|1011     |10204  |
|1006     |8883   |
|1012     |7429   |
|1005     |6391   |
|1013     |4322   |
|1004     |3022   |
|1014     |2277   |
|1003     |1201   |
|1015     |830    |
|1002     |294    |
|1016     |209    |
|1001     |96     |
|1017     |30     |
|1000     |8      |
|null     |0      |
+---------+-------+



In [20]:
fill_data_3 = fill_data_2.fillna(15193,"Barometer")

In [21]:
fill_data_3.select([functions.count( functions.when( functions.isnan(column) | functions.col(column).isNull(), column)).alias(column) for column in fill_data_3.columns]).show(truncate=False)

+-----------+----+---+--------+---+---+---+---+------+----+-------+----+---------+--------+---------+
|Flight_date|From|To |Duration|STD|ATD|STA|ATA|Signal|Temp|Weather|Wind|Direction|Humidity|Barometer|
+-----------+----+---+--------+---+---+---+---+------+----+-------+----+---------+--------+---------+
|0          |0   |0  |0       |0  |0  |0  |0  |0     |0   |5      |0   |0        |0       |0        |
+-----------+----+---+--------+---+---+---+---+------+----+-------+----+---------+--------+---------+



<h2>Transformation</h2>

In [22]:
def transform_signal(signal: str) -> int:
    if signal == 'red':
        return 1
    elif signal == 'green':
        return 2
    else: return 3
udf_signal = functions.udf(lambda row: transform_signal(row), IntegerType())

In [23]:
trans_data_1 = fill_data_3.withColumn("num_signal", udf_signal(functions.col("Signal"))).drop("Signal")

In [24]:
def transform_direc(dir: str):
    pattern = r'[0-9]+'
    result = re.findall(pattern, dir)
    return IntegerType().toInternal(int(result[0]))

In [25]:
udf_direct = functions.udf(transform_direc, IntegerType())

In [26]:
trans_data_2 = trans_data_1.withColumn("num_direct", udf_direct(functions.col("Direction"))).drop("Direction")
trans_data_2.show()

+-----------+--------------------+--------------+--------+-----+-----+-----+-----+----+--------------------+----+--------+---------+----------+----------+
|Flight_date|                From|            To|Duration|  STD|  ATD|  STA|  ATA|Temp|             Weather|Wind|Humidity|Barometer|num_signal|num_direct|
+-----------+--------------------+--------------+--------+-----+-----+-----+-----+----+--------------------+----+--------+---------+----------+----------+
| 13/09/2022|Ho Chi Minh City ...|Dong Hoi (VDH)|   01:15|15:30|15:46|17:05|17:01|  32|   Scattered clouds.|  13|      67|     1007|         2|       200|
| 12/09/2022|Ho Chi Minh City ...|Dong Hoi (VDH)|   01:09|18:00|18:17|19:20|19:26|  26|Thundershowers. P...|   6|      89|     1008|         2|         0|
| 11/09/2022|Ho Chi Minh City ...|Dong Hoi (VDH)|   01:13|15:30|16:23|17:05|17:36|  29|   Scattered clouds.|   6|      79|     1006|         3|         0|
| 09/09/2022|Ho Chi Minh City ...|Dong Hoi (VDH)|   01:14|15:30|17:07|

In [27]:
trans_data_2.select(functions.col("To")).distinct().show(25, False)

+----------------------+
|To                    |
+----------------------+
|Haiphong (HPH)        |
|Ho Chi Minh City (SGN)|
|Quang Ninh (VDO)      |
|Pleiku (PXU)          |
|Can Tho (VCA)         |
|Ca Mau (CAH)          |
|Hanoi (HAN)           |
|Con Dao (VCS)         |
|Qui Nhon (UIH)        |
|Da Nang (DAD)         |
|Dong Hoi (VDH)        |
|Nha Trang (CXR)       |
|Buon Ma Thuot (BMV)   |
|Tuy Hoa (TBB)         |
|Vinh (VII)            |
|Phu Quoc (PQC)        |
|Hue (HUI)             |
|Sao Vang (THD)        |
|Chu Lai (VCL)         |
|Da Lat (DLI)          |
|Dien Bien Phu (DIN)   |
+----------------------+



In [28]:
city_list = {'Haiphong (HPH)': 1, 'Quang Ninh (VDO)': 2, 'Pleiku (PXU)': 3, 'Can Tho (VCA)': 4,\
            'Ca Mau (CAH)': 5, 'Hanoi (HAN)': 6, 'Con Dao (VCS)': 7, 'Qui Nhon (UIH)': 8, 'Da Nang (DAD)': 9,\
            'Dong Hoi (VDH)': 10, 'Nha Trang (CXR)': 11, 'Buon Ma Thuot (BMV)': 12, 'Tuy Hoa (TBB)': 13,\
            'Vinh (VII)': 14, 'Phu Quoc (PQC)': 15, 'Hue (HUI)': 16, 'Sao Vang (THD)': 17, 'Chu Lai (VCL)': 18, 'Da Lat (DLI)': 19, 'Dien Bien Phu (DIN)': 20}

In [29]:
trans_data_2 = trans_data_2.filter(functions.col("To") != "Ho Chi Minh City (SGN)")

In [30]:
def transform_city(city:str) -> int:
    return IntegerType().toInternal(city_list[city])
udf_city = functions.udf(transform_city, IntegerType())

In [31]:
trans_data_3 = trans_data_2.withColumn("num_city", udf_city(functions.col("To"))).drop("To").drop("From")
trans_data_3.show()

+-----------+--------+-----+-----+-----+-----+----+--------------------+----+--------+---------+----------+----------+--------+
|Flight_date|Duration|  STD|  ATD|  STA|  ATA|Temp|             Weather|Wind|Humidity|Barometer|num_signal|num_direct|num_city|
+-----------+--------+-----+-----+-----+-----+----+--------------------+----+--------+---------+----------+----------+--------+
| 13/09/2022|   01:15|15:30|15:46|17:05|17:01|  32|   Scattered clouds.|  13|      67|     1007|         2|       200|      10|
| 12/09/2022|   01:09|18:00|18:17|19:20|19:26|  26|Thundershowers. P...|   6|      89|     1008|         2|         0|      10|
| 11/09/2022|   01:13|15:30|16:23|17:05|17:36|  29|   Scattered clouds.|   6|      79|     1006|         3|         0|      10|
| 09/09/2022|   01:14|15:30|17:07|17:05|18:21|  33|   Scattered clouds.|   6|      59|     1007|         1|         0|      10|
| 07/09/2022|   01:12|15:30|15:57|17:05|17:09|  27|Light rain. Broke...|  17|      89|     1007|        

In [32]:
def transform_time(_date: str, _time: str) -> int:
    _datetime = _date + " " +_time
    obj_datetime = datetime.strptime(_datetime, "%d/%m/%Y %H:%M")
    result = int(time.mktime(obj_datetime.timetuple()))
    return result
udf_time = functions.udf(transform_time, IntegerType())

In [33]:
trans_data_4 = trans_data_3.withColumn("tmp_STD", udf_time(functions.col("Flight_date"), functions.col("STD")))\
            .withColumn("tmp_ATD", udf_time(functions.col("Flight_date"), functions.col("ATD")))\
            .withColumn("tmp_STA", udf_time(functions.col("Flight_date"), functions.col("STA")))\
            .withColumn("tmp_ATA", udf_time(functions.col("Flight_date"), functions.col("ATA")))\
            .drop("STD").drop("ATD").drop("STA").drop("ATA").drop("Flight_date")
trans_data_4.show()

+--------+----+--------------------+----+--------+---------+----------+----------+--------+----------+----------+----------+----------+
|Duration|Temp|             Weather|Wind|Humidity|Barometer|num_signal|num_direct|num_city|   tmp_STD|   tmp_ATD|   tmp_STA|   tmp_ATA|
+--------+----+--------------------+----+--------+---------+----------+----------+--------+----------+----------+----------+----------+
|   01:15|  32|   Scattered clouds.|  13|      67|     1007|         2|       200|      10|1663057800|1663058760|1663063500|1663063260|
|   01:09|  26|Thundershowers. P...|   6|      89|     1008|         2|         0|      10|1662980400|1662981420|1662985200|1662985560|
|   01:13|  29|   Scattered clouds.|   6|      79|     1006|         3|         0|      10|1662885000|1662888180|1662890700|1662892560|
|   01:14|  33|   Scattered clouds.|   6|      59|     1007|         1|         0|      10|1662712200|1662718020|1662717900|1662722460|
|   01:12|  27|Light rain. Broke...|  17|      8

In [34]:
def transform_duration(_time: str) -> int:
    _hour = int(_time[:2])
    _minutes = int(_time[3:])
    return IntegerType().toInternal(_hour * 60 + _minutes)
udf_duration = functions.udf(transform_duration, IntegerType())

In [35]:
trans_data_5 = trans_data_4.withColumn("num_duration", udf_duration(functions.col("Duration"))).drop("Duration")

In [36]:
trans_data_5.show()

+----+--------------------+----+--------+---------+----------+----------+--------+----------+----------+----------+----------+------------+
|Temp|             Weather|Wind|Humidity|Barometer|num_signal|num_direct|num_city|   tmp_STD|   tmp_ATD|   tmp_STA|   tmp_ATA|num_duration|
+----+--------------------+----+--------+---------+----------+----------+--------+----------+----------+----------+----------+------------+
|  32|   Scattered clouds.|  13|      67|     1007|         2|       200|      10|1663057800|1663058760|1663063500|1663063260|          75|
|  26|Thundershowers. P...|   6|      89|     1008|         2|         0|      10|1662980400|1662981420|1662985200|1662985560|          69|
|  29|   Scattered clouds.|   6|      79|     1006|         3|         0|      10|1662885000|1662888180|1662890700|1662892560|          73|
|  33|   Scattered clouds.|   6|      59|     1007|         1|         0|      10|1662712200|1662718020|1662717900|1662722460|          74|
|  27|Light rain. Br

In [37]:
trans_data_5.groupBy("Weather").agg(functions.count("Weather").alias("num_row")).sort(functions.col("num_row").desc()).show(trans_data_5.count(),truncate=False)

+-------------------------------------------+-------+
|Weather                                    |num_row|
+-------------------------------------------+-------+
|Scattered clouds.                          |33155  |
|Passing clouds.                            |28019  |
|Partly sunny.                              |20616  |
|Broken clouds.                             |4621   |
|Fog.                                       |2329   |
|Partly cloudy.                             |1447   |
|Sprinkles. Scattered clouds.               |1202   |
|Thunderstorms. Scattered clouds.           |947    |
|Clear.                                     |823    |
|Light rain. Broken clouds.                 |773    |
|Sunny.                                     |637    |
|Sprinkles. Partly sunny.                   |614    |
|Light rain. Partly cloudy.                 |577    |
|More clouds than sun.                      |479    |
|Thunderstorms. Broken clouds.              |456    |
|Sprinkles. Broken clouds.  

from pyspark.sql.types import ArrayType

def split_weather_list(weather: str):
    _list = []
    for w in re.findall(r"([a-zA-Z]+[ \t]*[a-zA-Z]*).", weather):
        _list.append(w)
    return ArrayType(StringType()).toInternal(_list)
udf_createWList = functions.udf(split_weather_list, ArrayType(StringType()))

In [38]:
trans_data_6 = trans_data_5.fillna("Scattered clouds", subset="Weather")

In [39]:
trans_data_6.select("Weather").distinct().show()

+--------------------+
|             Weather|
+--------------------+
| Light rain. Cloudy.|
|Strong thundersto...|
|Thunderstorms. Sc...|
|Thundershowers. P...|
|Thunderstorms. Br...|
|              Sunny.|
|Rain showers. Pas...|
|Strong thundersto...|
|Lots of rain. Sca...|
|Thunderstorms. Pa...|
|Thundershowers. S...|
|               Mild.|
|Thunderstorms. Mo...|
|Strong thundersto...|
|Light rain. Scatt...|
|Rain showers. Clo...|
|       Rain. Cloudy.|
|Light rain. Mostl...|
|  Light rain. Clear.|
|Sprinkles. Overcast.|
+--------------------+
only showing top 20 rows



In [40]:
weather_dict = trans_data_6.select("Weather").distinct().to_pandas_on_spark().to_dict()

/home/hong_hai/.local/lib/python3.10/site-packages/pyspark/sql/dataframe.py:3313: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  warnings.warn(


In [41]:
def transweather(weather: str) -> int:
    for key, value in weather_dict["Weather"].items():
        if value == weather:
            return key
udf_weather = functions.udf(transweather, IntegerType())

In [42]:
trans_data_7 = trans_data_6.withColumn("num_weather", udf_weather(functions.col("Weather"))).drop("Weather")

In [43]:
trans_data_7.show()

+----+----+--------+---------+----------+----------+--------+----------+----------+----------+----------+------------+-----------+
|Temp|Wind|Humidity|Barometer|num_signal|num_direct|num_city|   tmp_STD|   tmp_ATD|   tmp_STA|   tmp_ATA|num_duration|num_weather|
+----+----+--------+---------+----------+----------+--------+----------+----------+----------+----------+------------+-----------+
|  32|  13|      67|     1007|         2|       200|      10|1663057800|1663058760|1663063500|1663063260|          75|         24|
|  26|   6|      89|     1008|         2|         0|      10|1662980400|1662981420|1662985200|1662985560|          69|          3|
|  29|   6|      79|     1006|         3|         0|      10|1662885000|1662888180|1662890700|1662892560|          73|         24|
|  33|   6|      59|     1007|         1|         0|      10|1662712200|1662718020|1662717900|1662722460|          74|         24|
|  27|  17|      89|     1007|         2|       260|      10|1662539400|1662541020|

<h1>Cai dat thuat toan</h1>

In [44]:
_df = trans_data_7.select("*").to_pandas_on_spark()

/home/hong_hai/.local/lib/python3.10/site-packages/pyspark/sql/dataframe.py:3313: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  warnings.warn(


In [45]:
_df = _df.sort_values(by=["num_direct","num_duration"])

training_df = _df.loc[:80000,:]
target_train = training_df["num_signal"]
training_data = training_df.drop(columns="num_signal")

testing_df = _df.loc[80000:,:]
target_test = testing_df.loc[:,"num_signal"]
testing_data = testing_df.drop(columns="num_signal")

In [46]:
def tinhKhoangCach(coor_1, coor_2):
    _tong = 0
    for i in range(12):
        _tong += (coor_2[i] - coor_1[i]) ** 2
    _tong =_tong ** (1/2)
    return _tong

In [47]:
def KNN(coor_1):
    k_nearest_value = []
    k_nearest_index = []
    for i, coor_2 in training_data.iterrows():
        temp = tinhKhoangCach(coor_1, coor_2)
        if len(k_nearest_value) < 10:
            k_nearest_value.append(temp)
            k_nearest_index.append(i)
        else:
            val_max = k_nearest_value[0]
            index_max = 0
            for k in range(len(k_nearest_value)):
                if k_nearest_value[k] > val_max:
                    val_max = k_nearest_value[k]
                    index_max = k
            if k_nearest_value[index_max] > temp:
                k_nearest_value[index_max] = temp
                k_nearest_index[index_max] = i
    red, green, yellow = 0, 0, 0
    for m in k_nearest_index:
        temp = target_train[m]
        if temp == 1:
            red += 1
        elif temp == 2:
            green += 1
        else: yellow += 1
    if red > green & red > yellow:
        return 1
    if green > red & green > yellow:
        return 2
    else: return 3


In [103]:
def testing():
    red_green = 0
    red_yellow = 0
    red_red = 0
    green_green = 0
    green_red = 0
    green_yellow = 0
    yellow_green = 0
    yellow_red = 0
    yellow_yellow = 0

    for i, coor in testing_data.iterrows():
        result = KNN(coor)
        target = target_test[i]
        if target == result:
            if result == 1:
                red_red += 1
            elif result == 2:
                green_green += 1
            else:
                yellow_yellow += 1
        else:
            if target == 1:
                if result == 2:
                    red_green += 1
                else:
                    red_yellow += 1
            if target == 2:
                if result == 1:
                    green_red += 1
                else:
                    green_yellow += 1
            else:
                if result == 1:
                    yellow_red += 1
                else:
                    yellow_green += 1
    print(red_green, red_yellow, red_red, "\n", green_green, green_yellow, green_red, "\n", yellow_green, yellow_yellow, yellow_red)

In [ ]:
testing()